### Modelado de Regresión y Clasificación

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    f1_score,
    roc_auc_score,
    average_precision_score
)

import sys

# Carga del script que contiene las rutinas comúnes para el modelado.
sys.path.append('../scripts')
import modelos_common as mcom

In [ ]:
# Se carga el dataset ya procesado y filtrado.
main_df = mcom.get_dataframe()

### Regresión

Para el análisis de regresión, se utilizan los modelos Ridge, RandomForestRegressor y HistGradientBoostingRegressor. Como baseline, se utiliza un DummyRegressor con strategy='mean'.

La estrategia es generar regresiones por cada cadena de supermercados. Es posible efectuar una única regresión para todo el dataset completo, pero para evitar que la influencia de variación de costos de una cadena de supermercados afecte la regresión de otro, se opta por hacer regreseiones separadas. Hay que tener presente que no todas las cadenas presentan la misma variabilidad en sus costos de productos.

Las variables a utilizar en la regresión son el producto_id, cadena_id y supermercado_id. Como ya las variables categóricas fueron codificadas con ids, no es necesario utilizar One-Hot Encoder. La variable objetivo es el costo. Esto lo podemos encontrar en la librería de rutinas comúnes (iter_regresion_Xy).

In [ ]:
metricas = []

# Se recorre cada cadena, junto con sus variables X, y objetivo y.
for cadena_name, X, y, _ in mcom.iter_regresion_Xy(main_df):
    # separación de datos de prueba y entrenamiento.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Se recorre todos los modelos de regresión contemplados para el análisis. Los modelos están definidos en la librería de rutinas comúnes.
    for reg_nombre, reg in mcom.ALL_REGRESION.items():
        modelo = Pipeline([
            ('scaler', StandardScaler()),
            ('regresion', reg),
        ])
        
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)
        
        # el modelo ajustado se guarda en disco para su posterior uso.
        mcom.guardar_modelo(modelo, 'regresion/{} - {}.joblib'.format(cadena_name, reg_nombre))
        
		# Cálculo de las métricas MAE, RMSE y WMAPE
        mae = mcom.calcular_mae(y_test, y_pred)
        rmse = mcom.calcular_rmse(y_test, y_pred)
        wmape = mcom.calcular_wmape(y_test, y_pred)
        
        metricas.append({
            'cadena': cadena_name,
            'regresion': reg_nombre,
            'MAE': mae,
            'RMSE': rmse,
            'WMAPE': wmape,
        })
    
metricas = pd.DataFrame(metricas)
display(metricas)    

### Clasificación

Para la clasificación se efectúa algo similar a la regresión. Se analizará en base a las cadenas de supermercados por separado. Se utilizarán los modelos LogisticRegression, RandomForestClassifier, HistGradientBoostingClassifier y DummyClassifier estratificado como baseline para mantener la misma distribución de clases como en la data de entrenamiento.

In [ ]:
metricas = []

for cadena_name, X, y, _ in mcom.iter_clasificacion_Xy(main_df):
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
    
    for cls_nombre, cls in mcom.ALL_CLASIFICACION.items():
        modelo = Pipeline([
            ('scaler', StandardScaler()),
            ('clasificador', cls),
        ])
        
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)
        
        # Se guarda el modelo en disco.
        mcom.guardar_modelo(modelo, 'clasificacion/{} - {}.joblib'.format(cadena_name, cls_nombre))
        
        # Algunos modelos generan probabilidades, por lo que hay que verificar cuáles lo hacen. En estos casos, se utiliza su salida de probabilidad.
        if hasattr(modelo.named_steps["clasificador"], "predict_proba"):
            y_proba = modelo.predict_proba(X_test)[:, 1]
        elif hasattr(modelo.named_steps["clasificador"], "decision_function"):
            y_proba = modelo.decision_function(X_test)
        else:
            y_proba = None
        
        # Cálculo de las métricas de evaluación.
        f1 = f1_score(y_test, y_pred, average='macro')
        roc_auc = None if y_proba is None else roc_auc_score(y_test, y_proba)
        pr_auc = None if y_proba is None else average_precision_score(y_test, y_proba)
        
        metricas.append({
            'cadena': cadena_name,
            'clasificador': cls_nombre,
            'F1': f1,
            'ROC-AUC': roc_auc,
            'PR-AUC': pr_auc,
        })

metricas = pd.DataFrame(metricas)
display(metricas)